Chapitre 9 - Premier formulaire et la recherche 
===

Nous savons désormais :

- Afficher des pages
- Récupérer des données

Mais le propre d'un site est d'être un peu plus interactif qu'un livre, et l'une des fonctions les plus recherchées est souvent la question.

## Quelques recommandations quant aux formulaires 

### Le formulaire en HTML

Les éléments nécessaires à un formulaire en HTML sont :

- la balise `<form />` qui doit englober l'ensemble des champs et des boutons
- on spécifiera la méthode à employer pour envoyer les données (attribut `method`)
- on spécifiera l'URL vers laquelle ces données sont envoyées (attribut `action`)
- la balise form inclut toujours un bouton via une balise `<button type="submit">Envoyer</button>`.
- on donnera toujours un `name` aux autres inputs
- on appréciera pour les champs de texte l'utilisation de l'attribut `placeholder` qui permet d'afficher une légende dans un champ

### Rappel : les types de champs textes

Depuis HTML5, les `<input type="text" />` peuvent être spécifiés plus finement :

- color
- date
- datetime-local
- email
- month
- number
- range
- search
- tel
- time
- url
- week

Pour plus de détails, nous vous recommandons la lecture de la page suivante : https://www.alsacreations.com/tuto/lire/1372-formulaires-html5-nouveaux-types-champs-input.html

### POST ou GET ?

Prenons un formulaire avec un champ `recherche`.

L'utilisation des méthodes POST ou GET a un impact visible sur les pages. En effet, **la méthode GET ajoute les informations du formulaire dans la barre URL**. Ainsi, l'utilisation d'un formulaire GET est fortement recommandée dans le cadre de formulaire de recherche. Il permettra de partager les résultats de recherche de manière simple !

Au contraire, **GET est à proscrire de tout formulaire à données sensibles**, on y préférera POST. Cela évitera d'avoir des adresses mails, ou pire, des mots de passe dans les adresses visitées. Il est en effet très simple de récupérer ce genre d'informations par des outils malicieux.

## Un formulaire, des formulaires ?

Dans le cadre d'un site tel que le nôtre, on peut imaginer assez facilement :

- une recherche rapide dans la barre de navigation en haut de la page
- un grand champ de recherche sur la page d'accueil
- une recherche à facettes

Dans un premier temps, nous allons nous concentrer sur les deux options simples.

### Dans la barre de recherche avec bootstrap

On utilisera le composant de recherche en barre de navigation de bootstrap pour réaliser un outil graphiquement acceptable. La [documentation](http://getbootstrap.com/docs/4.0/components/navbar/#forms) nous donne l'extrait suivant : 

```html
<nav class="navbar navbar-light bg-light justify-content-between">
  <a class="navbar-brand">Navbar</a>
  <form class="form-inline">
    <input class="form-control mr-sm-2" type="search" placeholder="Search" aria-label="Search">
    <button class="btn btn-outline-success my-2 my-sm-0" type="submit">Search</button>
  </form>
</nav>
```

On le modifiera afin de pouvoir réaliser notre propre formulaire :

#### Extrait du template container.html

```html
<nav class="navbar navbar-expand-md navbar-dark bg-dark justify-content-between">
    <a class="navbar-brand" href="{{ url_for('accueil') }}">Gazetteer</a>
    <form class="form-inline">
      <input class="form-control" type="search" name="keyword" placeholder="Recherche rapide" aria-label="Recherche">
      <button class="btn btn-outline-info" type="submit">Rechercher</button>
    </form>
</nav>
```

### Le HTML ok mais la récupération de l'information ?

Flask propose de récupérer les informations de formulaire via la variable `flask.request` (Documentation [simple](http://flask.pocoo.org/docs/0.12/quickstart/#accessing-request-data) / [complexe](http://flask.pocoo.org/docs/0.12/api/#incoming-request-data)). On commencera donc à importer cette variable :

```python
from flask import Flask, request
```

Cette variable possède entre autres un attribut `args` qui fonctionne comme un dictionnaire et qui récupère les valeurs disponibles dans l'URL de telle sorte que, pour l'URL `http://localhost/une_page?i=0&recherche=quelque%20chose` on aura :

```python
request.args == {
    "une_page": "0",
    "recherche": "quelque chose" # %20 remplace un espace dans les URLs
}
>>> True
```
*Au sujet des `%` dans les barres URL, voir https://en.wikipedia.org/wiki/Percent-encoding*

On va donc simplement créer une nouvelle route qui récupèrera cette information !

#### Extrait de `routes.py`
```python
@app.route("/recherche")
def recherche():
    # On préfèrera l'utilisation de .get() ici
    #   qui nous permet d'éviter un if long (if "clef" in dictionnaire and dictonnaire["clef"])
    motclef = request.args.get("keyword", None)
    # On crée une liste vide de résultat (qui restera vide par défaut
    #   si on n'a pas de mot clé)
    resultats = []
    # On fait de même pour le titre de la page
    titre = "Recherche"
    if motclef:
        resultats = Place.query.filter(
            Place.place_nom.like("%{}%".format(motclef))
        ).all()
        titre = "Résultat pour la recherche `" + motclef + "`"
    return render_template("pages/recherche.html", resultats=resultats, titre=titre)
```

#### Faire une requête `LIKE`

Pour construire une requête LIKE avec SQL Alchemy:

- On construit une requête à filtre `Place.query.filter()`
- Dans le filtre, on ajoute la propriété de champ (ici, `Place.place_nom`)
- On fait suivre ce champ de la méthode `.like()` avec le contenu de la recherche 
    - Ici les jokers `%` entourent la valeur de mot clef
- On récupère l'ensemble des résultats via `.all()`

#### Nouveau template : `templates/pages/recherche.html`

```html
{% extends "conteneur.html" %}

{% block titre %}| {{titre}}{%endblock%}

{% block corps %}
<h1>{{titre}}</h1>
    {% if resultats %}
    <p>Il y a {{resultats|length}} lieux qui répondent à votre requête :</p>
        <ul>
            {% for lieu in resultats %}
                <li><a href="{{url_for('lieu', place_id=lieu.place_id)}}">{{lieu.place_nom}}</a></li>
            {% endfor %}
        </ul>
    {% endif %}
{% endblock %}
```

#### `templates/container.html`

On corrige afin de metre l'action au formulaire :

```html
  <nav class="navbar navbar-expand-md navbar-dark bg-dark justify-content-between">
    <a class="navbar-brand" href="{{ url_for('accueil') }}">Gazetteer</a>
    <form class="form-inline" action="{{url_for("recherche")}}" method="GET">
      <input class="form-control" name="keyword" type="search" placeholder="Recherche rapide" aria-label="Recherche">
      <button class="btn btn-outline-info" type="submit">Rechercher</button>
    </form>
  </nav>
```

#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-flask/exemple13
python run.py
```

## Multiplier les formulaires

À partir du moment où la page de réception du formulaire est déjà disponible, on peut multiplier ce formulaire. Qu'importe le nombre d'entrées, il suffit d'une seule sortie !

Pour ce faire, on a créé un nouveau template `partials/recherche.html` que l'on inclura sur la page d'accueil et la page de recherche.

Regardez les templates de l'exemple14 puis lancez l'exemple.

#### Pour lancer l'exemple en terminal

On ouvrira un terminal, s'assurera d'être dans un environnement virtuel et on tapera depuis le dossier source

```sh
cd cours-flask/exemple14
python run.py
```

## Aller plus loin

Il existe une manière de bien sécuriser nos formulaires et d'en réutiliser facilement. Je vous conseille la lecture de [The Flask Mega-Tutorial Part III: Web Forms](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-iii-web-forms)